# **합성데이터 생성**

## **모델 불러오기**

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-AICA-5B'
# model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# 하단의 "가중치 로딩 리포트(경고)"는 텍스트만 생성할 경우 무시해도 됨

`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## **텍스트 생성함수**

In [2]:
def textAugmentation(model, PROMPT, FEWSHOT, topic, instruction):
    import time
    import torch

    # torch.cuda.synchronize()   # ✅ GPU 대기
    # start_time = time.time()

    instruction = f"""
        [{topic}] {instruction}
    """
    
    messages = [
        {"role": "system", "content": PROMPT},
        {"role": "user", "content": FEWSHOT},
        {"role": "user", "content": instruction},
    ]
    
    
    enc = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    
    
    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    outputs = model.generate(
        input_ids=enc["input_ids"],
        attention_mask=enc.get("attention_mask", None),
        
        # 길이 제어 (6줄 대화 최적)
        max_new_tokens=128,  
        
        # 샘플링 설정
        do_sample=True,
        temperature=0.6,
        top_p=0.9,

        # 반복 방지
        repetition_penalty=1.05,

        # 종료 / 패딩
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id
        # pad_token_id=tokenizer.eos_token_id  # 추가
    )
    
    prompt_len = enc["input_ids"].shape[-1]

    result = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True)

    # torch.cuda.synchronize()   # ✅ 다시 대기
    # end_time = time.time()
    
    # elapsed = end_time - start_time
    # print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")
    
    return result

## **프롬프트 작성**

In [3]:
PROMPT = """
너는 한국어 대화문을 생성하는 역할이다.

규칙:
1. 반드시 한국어로 작성한다.
2. 화자 이름은 쓰지 마.
3. 자연스럽고 상황에 맞는 말투(반말, 존댓말)로 대화해.
4. 6줄 이내로 작성한다.
5. "답변예시", "예시", "설명" 같은 문구를 절대 쓰지 않는다.
5. 예시의 문장, 표현을 그대로 사용 금지.
6. 예시에서 나온 고유 단어 사용 금지.
7. 대화문만 출력한다.
"""

FEWSHOT = """
질문 예:
[주말 계획] 이 키워드를 중심으로 답변예시의 형식만 참고해서 6줄의 한국어 대화문 1개 만들어줘.

답변 예:
이번 주말에 뭐 할까?
영화 한 편 보는 건 어때?
그럼 내가 예매할게.
"""

instruction = "이 키워드를 주제로 6줄의 한국어 대화문 1개 만들어줘."

In [4]:
# 테스트

print(
    textAugmentation(
        model=model,
        PROMPT=PROMPT,
        FEWSHOT=FEWSHOT,
        topic="책",
        instruction=instruction
    )
)

/tmp/ipykernel_11201/4166457105.py:33: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:347.)
  input_ids=enc["input_ids"],


IndexError: too many indices for tensor of dimension 2

## **키워드 추출**

In [5]:
def get_keyword(file_path):
    import csv
    from collections import Counter
    
    keyword_list = []
    with open(file_path, "r", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)  # 컬럼명 기준 접근
    
        for i, row in enumerate(reader):
            keywords = row["keywords"]   # keywords 컬럼 접근
            
            keyword_list += keywords.split("\n")

        counter = Counter(keyword_list)
        # results = [k for k, _ in counter.most_common(max_len)]
        
        return dict(counter)

In [6]:
import random

def sample_keywords(freq_dict, k=5000, alpha=0.5, min_freq=2):
    """
    freq_dict: {keyword: frequency}
    k       : 뽑을 키워드 개수 (중복 허용; Counter 기반이면 키 자체는 중복 없음)
    alpha   : 빈도 스무딩 지수 (0=균등 랜덤, 1=빈도 그대로, 0.5=제곱근 스무딩 추천)
    """
    # 빈도 필터링
    filtered = {
        k: v for k, v in freq_dict.items()
        if v >= min_freq
    }
    
    keywords = list(filtered.keys())
    freqs = list(filtered.values())

    # 빈도 스무딩(쏠림 완화)
    weights = [f ** alpha for f in freqs]

    # 가중치 기반 랜덤 샘플링
    return random.choices(keywords, weights=weights, k=k)

In [7]:
file_path = "../data/train_with_keywords.csv"

counter = get_keyword(file_path)
keyword_list = sample_keywords(counter, k=5000, alpha=0.5, min_freq=3)
print("전체 키워드 수: ", len(keyword_list))
print("중복 제거 키워드 수: ", len(set(keyword_list)))

전체 키워드 수:  5000
중복 제거 키워드 수:  711


## **기본 전처리**

### **전처리 항목**
- 줄바꿈("\n") -> 공백(" ")
- 쉼표(",") 제거
- 공백 정리

In [8]:
def clean_conversation(text: str) -> str:
    # 줄바꿈 제거 + 쉼표 제거 + 공백 정리
    text = text.replace("\r", " ").replace("\n", " ")
    text = text.replace(",", " ")
    text = " ".join(text.split())
    return text


## **파일 저장**

In [9]:
import csv
from pathlib import Path

def chunk_filename(base: str, chunk_id: int) -> str:
    # 옵션 A: 4자리 청크 번호
    return f"{base}_{chunk_id:03d}.csv"

def write_generations_chunked(
    model,
    PROMPT,
    FEWSHOT,
    instruction,
    out_dir="../data",
    base_name="general_conversations",
    chunk_size=1000,
    start_idx=0,
    start_chunk=1,
    keyword_list=[]
):
    Path(out_dir).mkdir(parents=True, exist_ok=True)

    idx = start_idx
    chunk_id = start_chunk
    in_chunk_count = 0

    f = None
    writer = None

    def open_new_chunk(chunk_id):
        path = Path(out_dir) / chunk_filename(base_name, chunk_id)
        f = open(path, "w", newline="", encoding="utf-8-sig")
        w = csv.writer(f)
        w.writerow(["idx", "class", "conversation"])
        return f, w, str(path)

    f, writer, current_path = open_new_chunk(chunk_id)


    
    for keyword in keyword_list:
        # 진행상황 표시
        if idx % chunk_size == 0:
            print("In progress:", current_path)
        
        gen = textAugmentation(model, PROMPT, FEWSHOT, keyword, instruction)
        
        conv = clean_conversation(gen)
        writer.writerow([idx, "일반", conv])

        idx += 1
        in_chunk_count += 1

        if in_chunk_count == chunk_size:
            f.close()
            chunk_id += 1
            in_chunk_count = 0  # chunk size까지 저장 후 다시 0으로 초기화
            print("Complete:", current_path)
            f, writer, current_path = open_new_chunk(chunk_id)
            
    if f:
        f.close()

    return idx, chunk_id  # 다음 이어쓰기용

In [10]:
idx, chunk_id = write_generations_chunked(
    model=model,
    PROMPT=PROMPT,
    FEWSHOT=FEWSHOT,
    instruction=instruction,
    out_dir="../data",
    base_name="general_conversations_with_keyword",
    chunk_size=1000,
    start_idx=0,
    start_chunk=1,
    keyword_list=keyword_list
)

In progress: ../data/general_conversations_with_keyword_001.csv
Complete: ../data/general_conversations_with_keyword_001.csv
In progress: ../data/general_conversations_with_keyword_002.csv
Complete: ../data/general_conversations_with_keyword_002.csv
In progress: ../data/general_conversations_with_keyword_003.csv
Complete: ../data/general_conversations_with_keyword_003.csv
In progress: ../data/general_conversations_with_keyword_004.csv
Complete: ../data/general_conversations_with_keyword_004.csv
In progress: ../data/general_conversations_with_keyword_005.csv
Complete: ../data/general_conversations_with_keyword_005.csv


## **8개 분류, 각각 10개 세부 주제 (ChapGPT 생성)**

In [11]:
import random

TOPICS = [
    # 음식·식사
    "점심 메뉴 못 정해서 고민",
    "혼밥 vs 같이 먹기",
    "배달앱 메뉴 추천 요청",
    "비 오는 날 음식 고르기",
    "다이어트 중 메뉴 고민",
    "야식 먹을지 말지 고민",
    "회사 근처 맛집 이야기",
    "집에 재료 없을 때 요리 고민",
    "카페 메뉴 고르기",
    "신메뉴 후기 공유",

    # 컨디션·생활
    "야근 후 너무 피곤함",
    "잠 설쳐서 졸림",
    "감기 기운 있음",
    "운동 시작했는데 힘듦",
    "허리나 목 통증",
    "스트레스 받아서 지침",
    "주말에 몰아서 잠 자기",
    "불면증 고민",
    "체력 떨어진 느낌",
    "건강검진 결과 이야기",

    # 약속·이동
    "약속 시간 변경 요청",
    "지각해서 사과 연락",
    "길 막혀서 늦음",
    "만날 장소 헷갈림",
    "어디쯤 왔는지 확인",
    "약속 취소 상황",
    "비 와서 약속 고민",
    "막차 걱정",
    "주차 자리 못 찾음",
    "귀가 중 통화",

    # 감정·관계
    "상사 때문에 스트레스",
    "친구와 싸운 후 고민",
    "연인과 오해 발생",
    "시험이나 면접 불안",
    "실패해서 우울함",
    "잘한 일 자랑",
    "위로 요청",
    "자신감 부족",
    "진로 고민",
    "번아웃 상태",

    # 소비·서비스
    "음식 배달 지연 항의",
    "상품 불량 교환 문의",
    "환불 규정 질문",
    "예약 변경 요청",
    "택배 분실 문의",
    "쿠폰 사용법 질문",
    "포인트 적립 문의",
    "영수증 재발급 요청",
    "가격 비교 상담",
    "AS 접수 문의",

    # 전화·메신저
    "부재중 전화 해명",
    "배터리 없어서 급통화",
    "통화 소리 끊김 문제",
    "끊긴 전화 재연결",
    "늦은 밤 전화 사과",
    "급하게 위치 물어봄",
    "운전 중 통화",
    "잘못 걸린 전화",
    "문자 vs 통화 선택",
    "통화 소리가 작은 문제",

    # 긴급·문제
    "지갑 분실",
    "휴대폰 분실",
    "교통사고 목격",
    "차 고장",
    "집 열쇠 분실",
    "길 잃음",
    "카드 분실 신고",
    "사기 의심 상황",
    "소음 신고",
    "위급 상황 신고",

    # 취미 생활
    "요즘 취미생활 뭐 하는지 이야기",
    "새로운 취미 시작 고민",
    "운동을 취미로 시작한 이야기",
    "게임을 취미로 즐기는 이야기",
    "드라마나 영화 보는 취미",
    "음악 듣기나 연주 취미 이야기",
    "책 읽는 취미 이야기",
    "사진이나 영상 찍는 취미",
    "요리나 베이킹 취미 이야기",
    "혼자 즐기는 취미생활 이야기"
]

topic_list = random.choices(TOPICS, k=5000)
len(topic_list)

5000

In [12]:
PROMPT = """
너는 한국어 대화문을 생성하는 역할이다.

규칙:
1. 반드시 한국어로 작성한다.
2. 화자 이름은 쓰지 마.
3. 자연스럽고 상황에 맞는 말투(반말, 존댓말)로 대화해.
4. 6줄 이내로 작성한다.
5. "답변예시", "예시", "설명" 같은 문구를 절대 쓰지 않는다.
5. 예시의 문장, 표현을 그대로 사용 금지.
6. 예시에서 나온 고유 단어 사용 금지.
7. 대화문만 출력한다.
"""

FEWSHOT = """
질문 예:
[주말 계획] 이것을 주제로 6줄의 한국어 대화문 1개 만들어줘.

답변 예:
이번 주말에 뭐 할까?
영화 한 편 보는 건 어때?
그럼 내가 예매할게.
"""

instruction = "이것을 주제로 6줄의 한국어 대화문 1개 만들어줘."

In [13]:
idx, chunk_id = write_generations_chunked(
    model=model,
    PROMPT=PROMPT,
    FEWSHOT=FEWSHOT,
    instruction=instruction,
    out_dir="../data",
    base_name="general_conversations",
    chunk_size=1000,
    start_idx=0,
    start_chunk=1,
    keyword_list=topic_list
)

In progress: ../data/general_conversations_001.csv
Complete: ../data/general_conversations_001.csv
In progress: ../data/general_conversations_002.csv
Complete: ../data/general_conversations_002.csv
In progress: ../data/general_conversations_003.csv
Complete: ../data/general_conversations_003.csv
In progress: ../data/general_conversations_004.csv
Complete: ../data/general_conversations_004.csv
In progress: ../data/general_conversations_005.csv
Complete: ../data/general_conversations_005.csv
